# Lab CMBLenses

## Model

In [1]:
] activate "/home/jaimerz/julia_repos/CMBLensingInferenceTestProblem"

  Activating project at `~/julia_repos/CMBLensingInferenceTestProblem`


In [2]:
using CMBLensing, CMBLensingInferenceTestProblem, CUDA, LaTeXStrings, MCMCDiagnosticTools, Plots, ProgressMeter, Random, Zygote
ENV["LINES"] = 10;

In [3]:
using Revise
using MCHMC

In [4]:
prob = load_cmb_lensing_problem(storage = Array,
                                T = Float32,
                                Nside = 4);

MAP_joint: 100%|████████████████████████████████████████| Time: 0:00:47
  step:        20
  logpdf:      377.00
  α:           0.008522608
  CG:          2 iterations
  Linesearch:  9 bisections


In [5]:
target = CMBLensingTarget(prob)

CMBLensingTarget(50, [-0.30936679244041443 0.0 … 0.0 0.0; 0.0 -0.8185905814170837 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 1.0], MCHMC.var"#nlogp#26"{CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem}(CMBLensingInferenceTestProblem.CMBLensingLogDensityProblem(BaseDataSet(d, Cf, Cn, Cn̂, M, M̂, B, B̂, logprior, Cϕ, Cf̃, D, G, L, Nϕ), [0.23362982273101807, 1.6145217418670654, -1.8872356414794922, -3.2767221927642822, 0.8644531965255737, 2.2458934783935547, -1.246073603630066, -2.639155864715576, 1.868626356124878, 3.251345157623291  …  -3.395840053599386e-9, 3.6597669339499816e-11, 4.677404241704153e-9, 1.211756028851596e-9, -5.810230518932258e-9, -2.3360031686792126e-9, 2.561251877253312e-9, -9.142808177209361e-10, -1.6094379124341003, 0.0], [0.0064222607761621475, 1.912354826927185, -2.338594675064087, -2.761254072189331, 0.3457961082458496, 2.8930203914642334, -1.2424980401992798, -3.303896427154541, 1.9643025398254395, 3.0344090461730957  …  -3.2332728494566254e-8, -3

## Sample

In [6]:
eps=0.00000005
L=sqrt(770)
spl=Sampler(;eps=eps, L=L)

Sampler(Settings(MersenneTwister(0), 5.0e-8, 27.748873851023216, 0.0, 0.1931833275037836, 0.0005, 2000, 1000, 10, "LF"), MCHMC.Leapfrog)

In [16]:
CMBLensing.LenseBasis(prob.Λmass * prob.Ωstart)

50-element Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}:
  1.0062761306762695
  6.856748104095459
 -7.938632011413574
  ⋮
 -1.6094379124341003
  0.0

In [17]:
 prob.Ωstart

50-element Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float32, 3}}, LambertMap{Array{Float32, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}:
  0.23362982273101807
  1.6145217418670654
 -1.8872356414794922
  ⋮
 -1.6094379124341003
  0.0

In [15]:
init = MCHMC.Get_initial_conditions(spl, target; initial_x=CMBLensing.LenseBasis(prob.Λmass * prob.Ωstart))

([1.0062761306762695, 6.856748104095459, -7.938632011413574, -13.803247451782227, 3.6748228073120117, 9.526881217956543, -5.246805191040039, -11.118765830993652, 7.926825523376465, 13.782153129577637  …  1.339787625e6, -1.74585575e6, -73665.875, 1.33907525e6, -3.75233725e6, 1.18877475e6, 2.68200575e6, -2.31747175e6, -1.6094379124341003, 0.0], [-0.3378972113132477, -0.26385048031806946, -0.033272042870521545, 0.025758985430002213, 0.23236389458179474, 0.14439669251441956, -0.13716626167297363, 0.12393136322498322, 0.15999744832515717, -0.10908547788858414  …  -0.04146997630596161, -0.10569348186254501, -0.028068387880921364, -0.15345917642116547, -0.1234259232878685, 0.1920272707939148, 0.06029362604022026, 0.131659135222435, -0.02143450868680486, 0.0552331932961709], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.8203425185902554e28, Inf], Inf, 0.0)

In [11]:
init, sample = MCHMC.Step(spl, target, init)
sample

50-element Field-(f°,ϕ°,θ)-Tuple{LambertQUMap{Array{Float64, 3}}, LambertMap{Array{Float64, 2}}, ComponentArrays.ComponentVector{Float64, Vector{Float64}, Tuple{ComponentArrays.Axis{(r = 1, Aϕ = 2)}}}}:
 -0.7551871548307489
 -1.9723189193323
  3.1538173487101595
  ⋮
 -1.6094379221181407
 -1.3095393417301704e-8

In [7]:
samples = Sample(spl, target; num_steps=10, initial_x=prob.Ωstart)

10-element Vector{Vector{Float64}}:
 [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 ⋮
 [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
 [NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN  …  NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]

## Plotting

In [8]:
Plots.default(fmt=:png, dpi=120, size=(500,300), legendfontsize=10)

In [16]:
plot([exp(Ω.θ.r) for (Ω,) in chain], label=L"r", xlabel="step")
plot!([exp(Ω.θ.Aϕ) for (Ω,) in chain], label=L"A_\phi")

LoadError: UndefVarError: `chain` not defined

In [17]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [18]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [19]:
plot(
    histogram([exp(Ω.θ.r) for (Ω,) in chain], xlabel=L"r", label=nothing, lw=1),
    histogram([exp(Ω.θ.Aϕ) for (Ω,) in chain], xlabel=L"A_\phi", label=nothing, lw=1)
)

LoadError: UndefVarError: `chain` not defined

In [20]:

ps = map([(:ϕ°,:I,L"L",L"\phi^\circ"), (:f°,:E,"L\ell",L"E^\circ"), (:f°,:B,L"\ell",L"B^\circ")]) do (k1, k2, xlabel, title)
    plot(get_Cℓ(prob.Ωtrue[k1][k2]); label="true", xlabel, title)
    plot!(get_Cℓ(prob.Ωstart[k1][k2]); label="start", xlabel, title)
    plot!(get_Cℓ(chain[end][1][k1][k2]); label="last sample", xlabel, title)
end
plot(ps..., layout=(1,3), xscale=:log10, yscale=:log10, size=(1000,300), legend=:bottomleft)

LoadError: UndefVarError: `prob` not defined